#  Call Dependencies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import backend as K

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import image

from keras.models import Sequential,Model,load_model
from keras.layers import Input, Dense, Activation, Dropout, LSTM, Flatten, Embedding, merge,TimeDistributed,concatenate,Bidirectional,BatchNormalization,Add,AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import metrics
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

from keras_bert import get_custom_objects
from tqdm import tqdm
from chardet import detect
import keras
from keras_bert import load_trained_model_from_checkpoint
from keras_bert import Tokenizer as k_Tokenizer

import sklearn
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,LabelBinarizer
from sklearn.externals import joblib
from sklearn.metrics import roc_auc_score,roc_curve,confusion_matrix,auc
from sklearn.utils import class_weight
from  sklearn.metrics import precision_recall_fscore_support


import os, argparse,cv2,h5py,string
import collections
from collections import Counter


from scipy import interp
from itertools import cycle
import seaborn as sns

from deepexplain.tensorflow import DeepExplain
import codecs

import innvestigate
import innvestigate.utils as iutils

from skimage.measure import compare_ssim as ssim
from skimage import data
from skimage.color import rgb2gray

from tensorflow.compat.v1 import ConfigProto,InteractiveSession
from keras.backend.tensorflow_backend import set_session,clear_session,get_session



config = tf.ConfigProto()
config.gpu_options.visible_device_list = "0"
config.gpu_options.per_process_gpu_memory_fraction =.6
set_session(tf.Session(config=config))

# config = tf.ConfigProto()
# config.gpu_options.visible_device_list = "1"
# config.gpu_options.per_process_gpu_memory_fraction = .3
# set_session(tf.Session(config=config))

# G =tf.Graph()
# sess1 = tf.Session(graph=G, config=tf.ConfigProto(log_device_placement=False,gpu_options=tf.GPUOptions(allow_growth=True,visible_device_list='0')))
# sess2 = tf.Session(graph=G, config=tf.ConfigProto(log_device_placement=False,gpu_options=tf.GPUOptions(allow_growth=True,visible_device_list='1')))



# Seed value

# Apparently you may use different seed values at each stage
seed_value= 2019

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
tf.set_random_seed(seed_value)


# Declare the two file name


In [ ]:
os.chdir('./Modality')  ### set working directory

#  Call the files

In [ ]:
### This files contains the Image ID, Related Question and Corrospoding answers. Each file contains training data (first 3200), validation data (next 500) and testing data (rest 125)

######################## If we do analysis for "Modality" category ############################


filename_mod="/home/local/AD/asarkar2/Question/C1_Modality_all.txt"
filename_glove="/home/local/AD/asarkar2/glove.6B.100d.txt"     ### golve vector

# filename_organ="/home/local/AD/asarkar2/Question/C3_Organ_all.txt"
# filename_plane="/home/local/AD/asarkar2/Question/C2_plane_all.txt"

### This files are uploaded to "Data file" in google drive
### Data File------> All Question  (Location)

####################### Image Path #################################################################

### This files are uploaded to "Data file" in google drive
### Data File------> All Image  (Location)

img_path_train="/home/local/AD/asarkar2/Image File/VQAMed2019_Train_Images/VQAMed2019_Train_Images"
img_path_valid="/home/local/AD/asarkar2/Image File/VQAMed2019_Valid_Images/VQAMed2019_Valid_Images"
img_path_test="/home/local/AD/asarkar2/Image File/VQAMed2019_Test_Images/VQAMed2019_Test_Images"






In [ ]:
os.getcwd()

#  Call a "separate" function to separate image id, question and answer from above file

In [ ]:
%run model_util.ipynb

In [ ]:
img_id=separate(filename_mod).get("Image_id")
question=separate(filename_mod).get("Question")
answer=separate(filename_mod).get("Answer")




In [ ]:
# first 3200 elements are from train data
# 3200-3700 elements are from valid data
# rest are from test data

image_id_train=img_id[:3200]
image_id_valid=img_id[3200:3700]
image_id_test=img_id[3700:]


question_train=question[:3200]
question_valid=question[3200:3700]
question_test=question[3700:]

answer_train=answer[:3200]
answer_valid=answer[3200:3700]
answer_test=answer[3700:]


# Call image_preprocess function to process the image file

In [ ]:
train_image=image_preprocess(img_path_train,image_id_train)
valid_image=image_preprocess(img_path_valid,image_id_valid)
test_image=image_preprocess(img_path_test,image_id_test)


In [ ]:
# train_image=np.load("./Image_array/modality_train_image.npy")
# valid_image=np.load("./Image_array/modality_valid_image.npy")
# test_image=np.load("./Image_array/modality_test_image.npy")



#  Standardized the image pixel

In [ ]:
train_image_std=train_image/255
valid_image_std=valid_image/255
test_image_std=test_image/255

#  Call label_preprocess Class to pre-process the answer (label)

In [ ]:

################# Integer Label Encoding ##########################################

integer_label_train=LabelPreprocess.integer_encoding(answer_train,answer[:3700])
integer_label_valid=LabelPreprocess.integer_encoding(answer_valid,answer[:3700])
integer_label_test=LabelPreprocess.integer_encoding(answer_test,answer[:3700])



In [ ]:
integer_label_train.shape

In [ ]:
################# One-Hot Label Encoding ##########################################

onehot_label_train=LabelPreprocess.onehot_encoding(answer_train,answer[:3700])
onehot_label_valid=LabelPreprocess.onehot_encoding(answer_valid,answer[:3700])
onehot_label_test=LabelPreprocess.onehot_encoding(answer_test,answer[:3700])


#  BERT model question pre-process

In [ ]:
train_ques_bert,valid_ques_bert,test_ques_bert=bert_question_preprocess(12,'/home/local/AD/asarkar2/BERT',question)

#  LSTM model question pre-process

In [ ]:
############################## sequence of the token for question ############################################

seq_train_question=QuestionPreprocess.sequence_question(question_train,question_train)
seq_valid_question=QuestionPreprocess.sequence_question(question_valid,question_train)
seq_test_question=QuestionPreprocess.sequence_question(question_test,question_train)


In [ ]:
############################### Padding question ########################################################

# maxlength of each question for modality-----> 11
# maxlength of each question for organ-----> 10
# maxlength of each question for modality-----> 9


padding_train_ques=QuestionPreprocess.padding_question(seq_train_question,maxlength=11,padding_criterion="post")
padding_valid_ques=QuestionPreprocess.padding_question(seq_valid_question,maxlength=11,padding_criterion="post")
padding_test_ques=QuestionPreprocess.padding_question(seq_test_question,maxlength=11,padding_criterion="post")


In [ ]:
padding_test_ques.shape

#  Embedding Matrix for Glove Vector

In [ ]:
def embedding(glove_filename,token_data): 
    word_index=token_data # call my_token function
    embeddings_index = dict()
    f = open(glove_filename,encoding='utf8') # call glove vector text file
    for line in f:
        values=line.split() # split the each line in text file
        word = values[0] # first index associate with word and othe other indexs represent embedding vector associated with that word 
        coefs = np.asarray(values[1:], dtype='float32') 
        embeddings_index[word] = coefs
    f.close()

    vocab_size=len(token_data)
    embedding_matrix=np.zeros((vocab_size+1,100)) # define embedding matrix
    print(word_index.items())
    for word,i in word_index.items():                # create our embedding matrix for each word of questions.
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix


In [ ]:
token_data=QuestionPreprocess.tokenize_question(question_train)

emb_mat=embedding(filename_glove,token_data)

In [ ]:
emb_mat.shape

# VQA Model 

In [ ]:
%run vgg_model_baseline.ipynb
%run resnet_model_baseline.ipynb
%run lstm_text_model.ipynb
%run bert_text_model.ipynb


In [ ]:
########################### VGG-16 & LSTM ####################

vgg=vgg_image_model()
lstm=lstm_text_model(emb_mat,emb_mat.shape[0],padding_train_ques.shape[1])

combined = Concatenate(axis=1)([lstm.output,vgg.output])
c1=Dense(2048,activation="relu")(combined)
c2=Dense(1000,activation="relu")(c1)
c3=Dense(onehot_label_train.shape[1])(c2)
c4=Activation("softmax")(c3)


model_vqa = Model(inputs=[lstm.input,vgg.input], outputs=c4)




In [ ]:
########################### VGG-16 & BERT ####################

vgg=vgg_image_model()
bert=bert_text_model(12,'/home/local/AD/asarkar2/BERT')

combined = Concatenate(axis=1)([bert.output,vgg.output])
c1=Dense(2048,activation="relu")(combined)
c2=Dense(1000,activation="relu")(c1)
c3=Dense(onehot_label_train.shape[1])(c2)
c4=Activation("softmax")(c3)


model_vqa = Model(inputs = bert.input+[vgg.input], outputs = c4)


In [ ]:
########################### RESNET-50 & LSTM ####################

resnet=resnet_image_model()
lstm=lstm_text_model(emb_mat,emb_mat.shape[0],padding_train_ques.shape[1])

combined = Concatenate(axis=1)([lstm.output,resnet.output])
c1=Dense(2048,activation="relu")(combined)
c2=Dense(1000,activation="relu")(c1)
c3=Dense(onehot_label_train.shape[1])(c2)
c4=Activation("softmax")(c3)


model_vqa = Model(inputs=[lstm.input,resnet.input], outputs=c4)


In [ ]:
########################### RESNET-50 & BERT ####################

resnet=resnet_image_model()
bert=bert_text_model(12,'/home/local/AD/asarkar2/BERT')

combined = Concatenate(axis=1)([bert.output,resnet.output])
c1=Dense(2048,activation="relu")(combined)
c2=Dense(1000,activation="relu")(c1)
c3=Dense(onehot_label_train.shape[1])(c2)
c4=Activation("softmax")(c3)


model_vqa = Model(inputs = bert.input+[resnet.input], outputs = c4)


#  Model Compile

In [ ]:
model_vqa.compile(loss='categorical_crossentropy', optimizer="SGD", metrics=["accuracy"])


#  Train Model

In [ ]:
#################################### VGG 16 & LSTM ###########################################################
callback = [EarlyStopping(monitor='val_loss',mode="min",verbose=1, patience=40),
             ModelCheckpoint('train_{}_{}.h5'.format("vgg","bert"), monitor='val_loss',mode="min" ,verbose=1,save_best_only=True)]
fit=model_vqa.fit([padding_train_ques,train_image_std],onehot_label_train,validation_data=([padding_valid_ques,valid_image_std],onehot_label_valid),epochs=300,batch_size=32,verbose=0, callbacks=callback)

In [ ]:
#################################### VGG 16 & BERT ###########################################################
callback = [EarlyStopping(monitor='val_loss',mode="min",verbose=1, patience=40),
             ModelCheckpoint('train_{}_{}.h5'.format("vgg","bert"), monitor='val_loss',mode="min" ,verbose=1,save_best_only=True)]
fit=model_vqa.fit(train_ques_bert+[train_image_std],onehot_label_train,validation_data=(valid_ques_bert+[valid_image_std],onehot_label_valid),epochs=300,batch_size=32,verbose=0, callbacks=callback)

#  Model Evaluation

In [ ]:
### Sample for any LSTM + (VGG/RESNET) model

################### load VGG 16 & LSTM model #################################

model = load_model('train_vgg_lstm.h5')

In [ ]:
train_accu=model.evaluate([padding_train_ques,train_image_std],onehot_label_train)
valid_accu=model.evaluate([padding_valid_ques,valid_image_std],onehot_label_valid)
test_accu=model.evaluate([padding_test_ques,test_image_std],onehot_label_test)


In [ ]:
print(train_accu, valid_accu,test_accu)

In [ ]:
### Sample for any BERT + (VGG/RESNET) model

################### load VGG 16 & BERT model #################################

model = load_model('train_vgg_bert.h5',custom_objects=get_custom_objects())

In [ ]:
train_accu=model.evaluate(train_ques_bert+[train_image_std],onehot_label_train)
valid_accu=model.evaluate(valid_ques_bert+[valid_image_std],onehot_label_valid)
test_accu=model.evaluate(test_ques_bert+[test_image_std],onehot_label_test)
